## 1. Imports
---

In [66]:
# imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Making Requests to GACC
---


#### Getting the list of available years from the selector object for the whole Monthly Bulletin Page

In [67]:
# setup base url for full monthly bulletin webpage
fullb_url = 'http://english.customs.gov.cn/statics/report/monthly.html'
# make a get request
fullb_response = requests.get(fullb_url)
# use Beautiful Soup to parse and create soup
fullb_soup = BeautifulSoup(fullb_response.text)


In [61]:
# find the list of permissible years

# find the list of years offered
select_years = fullb_soup.find('select', id='monthlysel')
# create and empty list
# append the values in our stripped strings to the list 
list_years = []
for value in select_years.stripped_strings:
    list_years.append(value)

# sort our list_years in place
list_years.sort()
    
# print out to show what is in there
print(list_years)

['2018', '2019', '2020', '2021', '2022']


The latest year is our base url. Previous years get appended to end of the url in the following format. 
So will need to append everything in our list of years onto the base url in a loop.  
http://english.customs.gov.cn/statics/report/monthly2021.html

#### Getting the lists of links to monthly aggregate data from Monthly Bulletin Page

In [ ]:
# setup base url for full monthly bulletin webpage
fullb_url = 'http://english.customs.gov.cn/statics/report/monthly.html'
# make a get request
fullb_response = requests.get(fullb_url)
# use Beautiful Soup to parse and create soup
fullb_soup = BeautifulSoup(fullb_response.text)

In [77]:
# finding the links for each monthly totals summary
monthly_total_links = fullb_soup.find_all('td')[3].find_all('a')

In [78]:
for i in range(len(monthly_total_links)):
    print(monthly_total_links[i].text, monthly_total_links[i]['href'])

 Jan. http://english.customs.gov.cn/Statics/bb598fd1-56f0-4e6e-b087-6237c12d310c.html
 Feb. http://english.customs.gov.cn/Statics/02029b0d-835a-4c8c-a006-7877b83e2e52.html
 Mar. http://english.customs.gov.cn/Statics/6d57fff0-92af-4adb-9e19-424ea8c41198.html


In [181]:
monthly_total_links[-1]['href']

'http://english.customs.gov.cn/Statics/6d57fff0-92af-4adb-9e19-424ea8c41198.html'

#### Getting the trade values from within the table on a given monthly trade link

Beginning with a random link (March 2022) from our found monthly totals links:  
http://english.customs.gov.cn/Statics/6d57fff0-92af-4adb-9e19-424ea8c41198.html

In [102]:
# setup month url
month_url = 'http://english.customs.gov.cn/Statics/6d57fff0-92af-4adb-9e19-424ea8c41198.html'

In [103]:
# make a get request to this first url
month_html_doc = requests.get(month_url)
# parse the html with beautiful soup
month_soup = BeautifulSoup(month_html_doc.content, 'html.parser')

In [120]:
# finding a table object from our soup object
table_object = month_soup.find('div', attrs={'class':'atcl-cnt'}).find('table')

In [182]:
# defining an empty pandas dataframe to collect out data
# give it column names only
df = pd.DataFrame(columns=['date', 'export', 'import'])

# delving into our table object

# find all the rows in our table object
for row in table.tbody.find_all('tr'):
    # find all data for each column in that row
    columns = row.find_all('td')
    
    # Now definte under which conditions we will add data to our dataframe
    
    # 1. Check it is not an empty list
    if columns != []:
        # 2. Check the length of the row (no columns) is greater than 4
        if len(columns) > 4:
            # 3. Check it's not the table header row
            if (columns[0].text.strip() != 'Year&Month'):
                # 4. And that date column isn't empty
                if (columns[0].text.strip() != ''):

                    # under these conditions, extract columns for date, exports & imports
                    
                    # remove commas so can convert to numeric
                    # change dot for hyphen in date for pd.datetime conversion
                    Date = columns[0].text.strip().replace('.','-')
                    Export = columns[2].text.strip().replace(',','')
                    Import = columns[3].text.strip().replace(',','')

                    # append these new values to our empty dataframe
                    df = df.append({'date':Date, 'export':Export, 'import':Import},
                                    ignore_index=True)

# perform some light pre-processing of our dataframe                    

# convert export and import columns to numeric
for col in df.columns[1:]:  # skip col 0, which is date
    df[col] = pd.to_numeric(df[col], errors='coerce')

# convert the date column to datetime and set as index
df['date'] = pd.to_datetime(df.date)

# set the date as the index, perform in place
# leave the dropping of old date col as true
df.set_index('date', drop=True, inplace=True)


In [184]:
df.tail()

,export,import
date,,
2021-11-01,324810,252997
2021-12-01,340431,246161
2022-01-01,327303,242237
2022-02-01,217533,187037
2022-03-01,276085,228704


## Getting the values for individual countries

#### Finding the monthly links for trade by partner country for a given year

In [264]:
 # setup base url for full monthly bulletin webpage
fullb_url = 'http://english.customs.gov.cn/statics/report/monthly2020.html'
# make a get request
fullb_response = requests.get(fullb_url)
# use Beautiful Soup to parse and create soup
fullb_soup = BeautifulSoup(fullb_response.text, 'html.parser')

In [265]:
# find the links to trade by partner country
by_partner_links = fullb_soup.find_all('td')[5].find_all('a')

In [266]:
for i in range(len(by_partner_links)):
    print(by_partner_links[i].text, by_partner_links[i]['href'])

 Feb. http://english.customs.gov.cn/Statics/80d0f985-3280-470a-9655-99d08989f181.html
 Mar. http://english.customs.gov.cn/Statics/fdd138b9-0683-41b5-a5d2-f6d9f972c356.html
 Apr. http://english.customs.gov.cn/Statics/1b8acf14-990b-4aef-9a82-a290c7d8fce8.html
 May. http://english.customs.gov.cn/Statics/e506a1cd-d147-49cd-ab43-4c45e117c2d3.html
 Jun. http://english.customs.gov.cn/Statics/065f4924-97df-4d63-96d9-cd3398b62ea1.html
 Jul. http://english.customs.gov.cn/Statics/f685730f-74c3-47a1-83d6-ba2969cf2f7a.html
 Aug. http://english.customs.gov.cn/Statics/d598d77d-ec33-424f-be31-8265194d08ad.html
 Sep. http://english.customs.gov.cn/Statics/43e4bf5a-94d5-489d-b0f8-698ec06bc67b.html
 Oct. http://english.customs.gov.cn/Statics/7f0503da-8220-4f44-8ebb-a4d75b29f44c.html
 Nov. http://english.customs.gov.cn/Statics/4eccd49c-b5b7-4096-821d-11fc6323392f.html
 Dec. http://english.customs.gov.cn/Statics/81bd4a20-01a8-45be-bcbc-f0627fd206da.html


#### Pulling the data out of a single month's trade by partner country

In [195]:
# use March 2022

# set the url to get
partner_url = 'http://english.customs.gov.cn/Statics/477e7bca-487f-4b6b-877e-b73a87108faa.html'
# use requests to retreive the response
partner_response = requests.get(partner_url)
# use BeautifulSoup to parse and create soup
partner_soup = BeautifulSoup(partner_response.text, 'html.parser')


In [206]:
# find the partner table object
partner_table = partner_soup.find('div', attrs={'class':'atcl-layout'}).find('table')

In [247]:
year = '2022' # will be available when looping through in our call

In [229]:
# create an empty dataframe
df = pd.DataFrame(columns=['partner', 'export', 'import'])

# find all the rows in our table object
for row in partner_table.tbody.find_all('tr'):
    # find all data for each column in that row
    columns = row.find_all('td')
    
    # set some conditions for adding into our df
    if columns != []:                          # not an empty list
        if len(columns) > 6:                   # if more than 6 columns
            if columns[0].text.strip() != '':  # removes empty rows
                if columns[0].text.strip().isnumeric() == False: # removes time label rows
        
                    # extract the partner name, exports and imports columns
                    # use relevant index of our columns object
                    # then take text part and strip. 
                    # for exports and imports, remove commas and replace any hyphens with zero
                    Partner = columns[0].text.strip()
                    Exports = columns[3].text.strip().replace(',','').replace('-','0')
                    Imports = columns[5].text.strip().replace(',','').replace('-','0')

                    # append these new values to our empty dataframe
                    df = df.append({'partner':Partner, 'export':Exports, 'import':Imports},
                                    ignore_index=True)

                    


In [244]:
# code for grabbing the month from the table
# get the rows, find the columns
# get the number contained in the first row to have a number in column 0
# break out of loop
for row in partner_table.tbody.find_all('tr'):
    columns = row.find_all('td')
    if columns[0].text.strip().isnumeric() == True:
        month = columns[0].text.strip()
        break

In [251]:
month_edit = month if len(month) == 2 else '0'+month
df['date'] = year+'-'+ month_edit

In [252]:
df

,partner,export,import,date
0,TOTAL,276084613,228704260,2022-03
1,Asia:,131493883,130322604,2022-03
2,Afghanistan,32000,1557,2022-03
3,Bahrian,177352,7084,2022-03
4,Bangladesh,2277953,93045,2022-03
...,...,...,...,...
245,Oth.Ocean.nes,6450,11,2022-03
246,Countries(reg.)unknown,0,116810,2022-03
247,ASEAN,41646300,33989506,2022-03
248,EU,44394685,24322434,2022-03


'228,704,260'

# OLD / NOT USED

In [3]:
# setup base url for gacc web page
prelim_url = 'http://english.customs.gov.cn/statics/report/preliminary.html'
# make a get request
prelim_html_doc = requests.get(prelim_url)
# parse the html with breautiful soup
prelim_soup = BeautifulSoup(prelim_html_doc.content, 'html.parser')

In [ ]:
# print out the prettified soup
print(prelim_soup.prettify)

In [8]:
# command for finding list of exports & imports in USD
name_href_list = prelim_soup.find_all('td')[3].find_all('a')
for i in range(len(name_href_list)):
    print(name_href_list[i].text, name_href_list[i]['href'])

 Feb. http://english.customs.gov.cn/Statics/26047d08-f2cd-4483-9023-9e89b5ca1686.html
 Mar. http://english.customs.gov.cn/Statics/991ba64c-f99e-4b47-8557-cc178cb2a14c.html


In [42]:
month_url = 'http://english.customs.gov.cn/Statics/991ba64c-f99e-4b47-8557-cc178cb2a14c.html'
march_r = requests.get(month_url)
march_soup = BeautifulSoup(march_r.content, 'html.parser')

In [50]:
exports = march_soup.find_all('b')[15]
imports = march_soup.find_all('b')[20]

In [51]:
month_url = 'http://english.customs.gov.cn/Statics/7043b548-0b1c-4251-83eb-f1aca196e612.html'
march_r = requests.get(month_url)
march_soup = BeautifulSoup(march_r.content, 'html.parser')